In [1]:
!git clone https://github.com/rndmit/crossway.git

Cloning into 'crossway'...
remote: Enumerating objects: 411, done.
remote: Counting objects: 100% (411/411), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 411 (delta 148), reused 393 (delta 137), pack-reused 0
Receiving objects: 100% (411/411), 7.18 MiB | 9.90 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 12.3MB/s 


In [4]:
#@title
from sklearn.metrics import confusion_matrix
from skimage import measure
from PIL import Image, ImageDraw
import argparse
import datetime
import json
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_addons as tfa
import test
import time
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import yaml
import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from scipy.special import expit
import matplotlib.pyplot as plt
import random

In [5]:
np.set_printoptions(precision=2, threshold=10000, suppress=True, linewidth=10000)

In [6]:
obj="car"
run=obj+""
percentage=0.6

sample_dir="./crossway/sample_data"
dataset_dir=sample_dir+"/dataset"
image_dir=sample_dir + "/training"
train_image_dir="./crossway/sample_data/training"
test_image_dir="./crossway/sample_data/test"
ckpt_dir = "./crossway/ckpts/"+run
batch_size = 4
width=1280
height=720
patch_width_height=256

base_filter_size=16
learning_rate=0.001
pos_weight=5.0
steps=100
secs=1000
train_steps=100
use_skip_connections=False

In [7]:
#@title
with open(sample_dir + '/labels.json') as json_file:
    labels = json.load(json_file)

if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)

In [8]:
#@title
def read_dataset():
  imgs = list(filter(lambda p: not (p.endswith(".png") or p == "res"),  os.listdir(dataset_dir)))
  random.shuffle(imgs)
  border = int(len(imgs)*percentage)
  training_imgs = imgs[:border]
  test_imgs = imgs[:border]

  train_imgs_xys_bitmaps = img_xys_iterator(
    dataset_dir,
    training_imgs,
    batch_size=batch_size,
    patch_width_height=256,
    distort_rgb=True,
    flip_left_right=False,
    random_rotation=True,
    repeat=True,
    width=None,
    height=None
  )

  test_imgs_xys_bitmaps = img_xys_iterator(
    dataset_dir,
    test_imgs,
    batch_size=batch_size,
    patch_width_height=None,
    distort_rgb=False,
    flip_left_right=False,
    random_rotation=False,
    repeat=False,
    width=width,
    height=height
  )
  return (train_imgs_xys_bitmaps, len(training_imgs), test_imgs_xys_bitmaps, len(test_imgs))

def img_xys_iterator(image_dir, imgs_path, batch_size, patch_width_height, distort_rgb,
                    flip_left_right, random_rotation, repeat, width, height,
                    label_rescale=0.5):
    width_height_set = False
    if width is not None or height is not None:
      if width is None or height is None:
        raise Exception("when setting --width or --height must set both")
      width_height_set = True
    patch_set = patch_width_height is not None
    if patch_set and width_height_set:
      raise Exception("need to set either --patch-width-height or --width and --height, not both")
    if not (patch_set or width_height_set):
      raise Exception("need to set one of either --patch-width-height or both --width and --height")
      
    rgb_filenames = []
    bitmap_filenames = []
    for fname in imgs_path:
        rgb_filename = "%s/%s" % (image_dir, fname)
        rgb_filenames.append(rgb_filename)
        bitmap_filename = "%s/%s.png" % (image_dir, fname.replace(".jpg", ""))
        # print("rgb", rgb_filename, "bitmap", bitmap_filename)
        if not os.path.isfile(bitmap_filename):
            raise Exception("label bitmap img [%s] doesn't exist for training example [%s]. did you run materialise_label_db.py?"
                      % (bitmap_filename, fname))
        bitmap_filenames.append(bitmap_filename)
    
    def decode_images(rgb_filename, bitmap_filename):
        rgb = tf.image.decode_image(tf.io.read_file(rgb_filename))
        rgb = tf.cast(rgb, tf.float32)
        rgb = (rgb / 127.5) - 1.0
        bitmap = tf.image.decode_image(tf.io.read_file(bitmap_filename))
        bitmap = tf.cast(bitmap, tf.float32)
        bitmap /= 256
        return rgb, bitmap
    
    def random_crop(rgb, bitmap):
      patch_width = patch_height = patch_width_height
      height, width = tf.shape(rgb)[0], tf.shape(rgb)[1]
      offset_height = tf.random.uniform([], 0, height - patch_height, dtype=tf.int32)
      offset_width = tf.random.uniform([], 0, width-patch_width, dtype=tf.int32)
      rgb = tf.image.crop_to_bounding_box(rgb, offset_height, offset_width, patch_height, patch_width)
      rgb = tf.reshape(rgb, (patch_height, patch_width, 3))
      bitmap = tf.image.crop_to_bounding_box(bitmap,
                                           tf.cast(tf.cast(offset_height, tf.float32) * label_rescale, tf.int32),
                                           tf.cast(tf.cast(offset_width, tf.float32) * label_rescale, tf.int32),
                                           int(patch_height * label_rescale), int(patch_width * label_rescale))
      bitmap = tf.reshape(bitmap, (int(patch_height * label_rescale), int(patch_width * label_rescale), 3))
      bitmap = bitmap[:,:,:2]
      return rgb, bitmap

    def augment(rgb, bitmap):
      if flip_left_right:
        random = tf.random.uniform([], 0, 1, dtype=tf.float32)
        rgb, bitmap = tf.cond(random < 0.5, 
                              lambda: (rgb, bitmap),
                              lambda: (tf.image.flip_left_right(rgb),
                                       tf.image.flip_left_right(bitmap)))
      if distort_rgb:
        rgb = tf.image.random_brightness(rgb, 0.1)
        rgb = tf.image.random_contrast(rgb, 0.9, 1.1)
        rgb = tf.clip_by_value(rgb, clip_value_max=1.0, clip_value_min=-1.0)

      if random_rotation:
        random_rotation_angle = tf.random.uniform([], -0.4, 0.4, dtype=tf.float32)
        rgb, bitmap = (tfa.image.rotate(rgb, random_rotation_angle, 'BILINEAR'),
                       tfa.image.rotate(bitmap, random_rotation_angle, 'BILINEAR'))
        return rgb, bitmap
      
    def set_explicit_size(rgb, bitmap):
      bmap = tf.reshape(bitmap, (height // 2, width // 2, 3))
      bmap = bitmap[:,:,:2]
      return (tf.reshape(rgb, (height, width, 3)),
              bmap)
    
    dataset = tf.data.Dataset.from_tensor_slices((
        tf.constant(rgb_filenames),
        tf.constant(bitmap_filenames)
    ))

    dataset = dataset.map(decode_images, num_parallel_calls=8)

    if repeat:
      if len(rgb_filenames) < 1000:
        dataset = dataset.cache()
      dataset = dataset.shuffle(1000).repeat()

    if patch_width_height is not None:
      dataset = dataset.map(random_crop, num_parallel_calls=8)
    else:
      dataset = dataset.map(set_explicit_size, num_parallel_calls=8)

    if flip_left_right or distort_rgb or random_rotation:
      dataset = dataset.map(augment, num_parallel_calls=8)

    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
      

In [9]:
#@title
def zero_centered_array_to_pil_image(orig_array):
    assert orig_array.dtype == np.float32
    h, w, c = orig_array.shape
    assert c == 3
    array = orig_array + 1  # 0.0 -> 2.0
    array *= 127.5  # 0.0 -> 255.0
    array = array.copy().astype(np.uint8)
    assert np.min(array) >= 0
    assert np.max(array) <= 255
    return Image.fromarray(array)

def bitmap_to_pil_image(bitmap):
  assert bitmap.dtype == np.float32
  h, w, c = bitmap.shape
  assert c == 1
  rgb_array = np.zeros((h, w, 3), dtype=np.uint8)
  single_channel = bitmap[:,:,0] * 255
  rgb_array[:,:,0] = single_channel
  rgb_array[:,:,1] = single_channel
  rgb_array[:,:,2] = single_channel
  return Image.fromarray(rgb_array)

def side_by_side(rgb, bitmap):
  h, w, _ = rgb.shape
  canvas = Image.new('RGB', (w*2, h), (50, 50, 50))
  lhs = zero_centered_array_to_pil_image(rgb)
  canvas.paste(lhs, (0, 0))
  # paste bitmap version of labels on right hand side
  # black with white dots at labels
  rhs = bitmap_to_pil_image(bitmap)
  rhs = rhs.resize((w, h))
  canvas.paste(rhs, (w, 0))
  # draw on a blue border (and blue middle divider) to make it
  # easier to see relative positions.
  draw = ImageDraw.Draw(canvas)
  draw.polygon([0,0,w*2-1,0,w*2-1,h-1,0,h-1], outline='blue')
  draw.line([w,0,w,h], fill='blue')
  canvas = canvas.resize((w, h//2))
  return canvas

Materialize labels

In [10]:
#@title
def xys_to_bitmap(xys, height, width, rescale=1.0, prescale=0.0):
    # bitmap = np.zeros((int(height*rescale), int(width*rescale), 3), dtype=np.uint8)
    img = Image.new('RGB', (int(width*rescale), int(height*rescale)))
    canvas = ImageDraw.Draw(img)
    for i in range(2):
      for x, y in xys[i]:
          scale=prescale[i]
          coords = (int(x*rescale-scale), int(y*rescale-scale), int(x*rescale+scale), int(y*rescale+scale))
          # print("coords",coords, "i", i)
          if i == 0:
            canvas.rectangle(coords, fill='red')
          if i == 1:
            canvas.rectangle(coords, fill='green')
          # bitmap[int(y*rescale), int(x*rescale), i] = 1.0
    bitmap = np.array(img)
    return bitmap
def get_point_from_bbox(x,y,width,height):
    return (x+width/2,y+height/2)
def get_points(flabels):
    labels = [[],[]]
    for o in flabels["Label"]["objects"]:
        bbox = o["bbox"]
        label = tuple(get_point_from_bbox(bbox['left'], bbox['top'], bbox['width'], bbox['height']))
        if o["value"] == "human":
          labels[0].append(label)
        else:
          labels[1].append(label)
    return labels
def bitmap_to_pil_image(bitmap):
    # h, w, c = bitmap.shape
    # bitmap = np.uint8(bitmap[:,:,:] * 255)
    return Image.fromarray(bitmap, mode='RGB')
def materialize_labels(labels, rescale=[1.0, 4.0]):
  for i, frame in enumerate(labels):
      xys = get_points(frame)
      bitmap = xys_to_bitmap(xys, 720, 1280, 0.5, rescale)
      single_chan_img = bitmap_to_pil_image(bitmap)
      single_chan_img.save("%s/%s.png" % (dataset_dir, frame["ID"]))

In [11]:
materialize_labels(labels, [0.5, 7.0])

Train model

In [12]:
#@title
def contruct_model(width, height, base_filter_size, use_batch_norm=True, use_skip_connections=True):
  def conv_bn_relu_block(i, name, filters, strides):
    o = Conv2D(filters=filters, kernel_size=3, strides=strides, padding='same')(i)
    o = BatchNormalization()(o)
    return ReLU()(o)
  inputs = Input(shape=(height, width, 3), name='inputs')
  e1 = conv_bn_relu_block(inputs, 'e1', filters=base_filter_size, strides=2)
  e2 = conv_bn_relu_block(e1, 'e2', filters=2*base_filter_size, strides=2)
  e3 = conv_bn_relu_block(e2, 'e3', filters=4*base_filter_size, strides=2)
  e4 = conv_bn_relu_block(e3, 'e4', filters=8*base_filter_size, strides=2)

  d1 = UpSampling2D(name='e4nn')(e4)
  if use_skip_connections:  
    d1 = Concatenate(name='d1_e3')([d1, e3])
  d1 = conv_bn_relu_block(d1, 'd1', filters=4*base_filter_size, strides=1)

  d2 = UpSampling2D(name='d1nn')(d1)
  if use_skip_connections:
    d2 = Concatenate(name='d2_e2')([d2, e2])
  d2 = conv_bn_relu_block(d2, 'd2', filters=2*base_filter_size, strides=1)

  d3 = UpSampling2D(name='d2nn')(d2)
  if use_skip_connections:
    d3 = Concatenate(name='d3_e1')([d3, e1])
  d3 = conv_bn_relu_block(d3, 'd3', filters=base_filter_size, strides=1)

  # d4 = UpSampling2D(name='d3nn')(d3)
  # d4 = conv_bn_relu_block(d3, 'd4', filters=base_filter_size, strides=1)

  logits = Conv2D(filters=2, kernel_size=1, strides=1, activation=None, name='logits')(d3)

  return Model(inputs=inputs, outputs=logits)

def compile_model(model, learning_rate, pos_weight=1.0):
  def weighted_xent(y_true, y_predicted):
    return tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(labels=y_true, logits=y_predicted, pos_weight=pos_weight))
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),loss=weighted_xent)
  return model

def fit_model(train_model, test_model):
  start_time=time.time()
  done=False
  step=0

  while not done:
    history = train_model.fit(train_imgs_xys_bitmaps, epochs=1, verbose=1, steps_per_epoch=train_steps)
    train_loss=history.history['loss'][0]
    
    test_model.set_weights(train_model.get_weights())
    test_loss = test_model.evaluate(test_imgs_xys_bitmaps, verbose=1, steps=num_test_steps)

    dts = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    save_filename = "%s/%s" % (ckpt_dir, dts)
    train_model.save_weights(save_filename)

    log = []
    log.append("step %d/%d" % (step, steps))
    log.append("time %d" % int(time.time()-start_time))
    log.append("train_loss %f" % train_loss)
    log.append("test_loss %s" % test_loss)
    print("\t".join(log))

    step += 1
    if step >= steps:
      done = True

In [13]:
(train_imgs_xys_bitmaps, num_train_files, test_imgs_xys_bitmaps, num_test_files) = read_dataset()
num_test_steps = num_test_files // batch_size
print("num_test_files=", num_test_files, "batch_size=", batch_size, "=> num_test_steps=", num_test_steps)

train_model = contruct_model(width=patch_width_height, height=patch_width_height, base_filter_size=base_filter_size)
compile_model(train_model, learning_rate=learning_rate, pos_weight=pos_weight)
test_model = contruct_model(width=width, height=height, base_filter_size=base_filter_size)
compile_model(test_model, learning_rate=learning_rate, pos_weight=pos_weight)

fit_model(train_model, test_model)

num_test_files= 106 batch_size= 4 => num_test_steps= 26
26/26 [==============================] - 2s 51ms/step - loss: 0.3921
step 0/100	time 13	train_loss 0.485642	test_loss 0.391886830329895
26/26 [==============================] - 2s 50ms/step - loss: 0.2126
step 1/100	time 17	train_loss 0.243677	test_loss 0.21261809766292572
26/26 [==============================] - 2s 51ms/step - loss: 0.1329
step 2/100	time 21	train_loss 0.147524	test_loss 0.13287712633609772
26/26 [==============================] - 2s 48ms/step - loss: 0.0816
step 3/100	time 25	train_loss 0.095394	test_loss 0.0816221758723259
26/26 [==============================] - 2s 53ms/step - loss: 0.0629
step 4/100	time 29	train_loss 0.073404	test_loss 0.0629178062081337
26/26 [==============================] - 2s 51ms/step - loss: 0.0568
step 5/100	time 33	train_loss 0.060324	test_loss 0.05680389702320099
26/26 [==============================] - 2s 49ms/step - loss: 0.0626
step 6/100	time 37	train_loss 0.052603	test_loss 0.

In [14]:
def restore_model():
  model = contruct_model(width=width, height=height, base_filter_size=base_filter_size)
  checkpoint_info = yaml.load(open("%s/checkpoint" % ckpt_dir).read())
  latest_ckpt = checkpoint_info['model_checkpoint_path']
  model.load_weights("%s/%s" % (ckpt_dir, latest_ckpt))
  return model

In [15]:
def centroids_of_connected_components(bitmap, threshold=0.05, rescale=1.0):
  mask = bitmap > threshold
  bitmap = np.zeros_like(bitmap)
  bitmap[mask] = 1.0
  all_labels = measure.label(bitmap)
  centroids = []
  for region in measure.regionprops(label_image=all_labels):
    cx, cy = map(lambda p: int(p*rescale), (region.centroid[0], region.centroid[1]))
    centroids.append((cx, cy))
  return centroids

Predict

In [16]:
model = restore_model()

In [17]:
!mkdir /content/crossway/sample_data/dataset/res

In [18]:
imgs = list(filter(lambda p: not (p.endswith(".png") or p == "res"),  os.listdir(dataset_dir)))
for idx, filename in enumerate(sorted(imgs)):
  img = np.array(Image.open(dataset_dir+"/"+filename), dtype=np.float32)
  img = (img / 127.5) - 1.0
  prediction = expit(model.predict(np.expand_dims(img, 0))[0])
  h, w, c = prediction.shape
  rgb_array = np.zeros((h,w,3), dtype=np.uint8)
  rgb_array[:,:,0] = prediction[:,:,0] * 255
  rgb_array[:,:,1] = prediction[:,:,1] * 255
  # print("\t".join(map(str, [idx, filename])))
  pred_img = Image.fromarray(rgb_array)
  pred_img.save("%s/res/%s_debug.png" % (dataset_dir, filename.replace(".jpg", "")))

0	ckl0tqscn00003b69n2slxwhc.jpg
1	ckl0tqy0i00003b69u3wxso4p.jpg
2	ckl0tx7ck00013b69xqgcld5y.jpg
3	ckl0tzvas00003b6dwebqfnth.jpg
4	ckl0u29v300023b691ijolk8k.jpg
5	ckl0u2h7j00003669ehs1sl90.jpg
6	ckl0u907z00033b69dt8230c6.jpg
7	ckl0uczwm00013669r8i9srq3.jpg
8	ckl0udhy800043b69kqqz85gb.jpg
9	ckl0udjnw00003b6d8ctd4nrb.jpg
10	ckl0uh9lw00013b694tuvpoxq.jpg
11	ckl0uic9y00053b69mw2hyi1d.jpg
12	ckl0ujgjy00023669nkipux5t.jpg
13	ckl12f1e200003b69qpdzdlmp.jpg
14	ckl12ihy400013b69785orwgc.jpg
15	ckl12l5o400023b690wyrf35l.jpg
16	ckl12nyb800033b690yyjdox1.jpg
17	ckl13h69800043b69988tvosu.jpg
18	ckl13lc0v00053b69ni5ed2cg.jpg
19	ckl13nxxd00063b69aj29f62k.jpg
20	ckl13qt3y00073b69v0xum2qr.jpg
21	ckl13tlkv00083b69qyuocjkb.jpg
22	ckl13xfum00093b69xzw5fyq6.jpg
23	ckl140n7l000a3b69dx7wc9n7.jpg
24	ckl142q7h000b3b69z7q2xk44.jpg
25	ckl146dkr000c3b69slllz6xe.jpg
26	ckl1498rq000d3b69x5f3ummb.jpg
27	ckl14jqyy00003b6985pwkig0.jpg
28	ckl14lbg500013b69z2m1s7mb.jpg
29	ckl14nf4u00023b69pa12bcvi.jpg
30	ckl14pccy00033b69

In [19]:
polygons = [
  [(180, 392), (269, 313), (360, 320), (288, 404)],
  [(372, 321), (315, 398), (808, 404), (812, 321)],
  [(817, 281), (1015, 286), (1026, 422), (818, 404)],
  [(744, 418), (990, 434), (1029, 720), (702, 720)],
  [(1132, 444), (1280, 436), (1280, 558), (1169, 583)]
]
pgons = list(map(lambda p: Polygon(p), polygons) )

In [26]:
imgs = list(filter(lambda p: not (p.endswith(".png") or p == "res"),  os.listdir(dataset_dir)))
for idx, filename in enumerate(sorted(imgs)):
  img = np.array(Image.open(dataset_dir+"/"+filename), dtype=np.float32)
  img = (img / 127.5) - 1.0
  prediction = expit(model.predict(np.expand_dims(img, 0))[0])

  h_centroids = centroids_of_connected_components(prediction[:,:,0], threshold=0.1,rescale=2.0)
  # print("\t".join(map(str, [idx, filename])))
  debug_img = zero_centered_array_to_pil_image(img)
  canvas = ImageDraw.Draw(debug_img)
  for y, x in h_centroids:
    point = Point(x, y)
    for p in pgons:
      if p.contains(point):
        canvas.rectangle((x-4, y-4, x+4, y+4), fill='blue')
      else:
        canvas.rectangle((x-2, y-2, x+2, y+2), fill='red')
  c_centroids = centroids_of_connected_components(prediction[:,:,1], threshold=0.4,rescale=2.0)
  for y, x in c_centroids:
    point = Point(x, y)
    for p in pgons:
      if p.contains(point):
        canvas.rectangle((x-4, y-4, x+4, y+4), fill='blue')
      else:
        canvas.rectangle((x-2, y-2, x+2, y+2), fill='yellow')
  for xy in polygons:
    canvas.polygon(xy, outline="wheat")
  debug_img.save("%s/res/%s.png" % (dataset_dir, filename))

0	ckl0tqscn00003b69n2slxwhc.jpg
1	ckl0tqy0i00003b69u3wxso4p.jpg
2	ckl0tx7ck00013b69xqgcld5y.jpg
3	ckl0tzvas00003b6dwebqfnth.jpg
4	ckl0u29v300023b691ijolk8k.jpg
5	ckl0u2h7j00003669ehs1sl90.jpg
6	ckl0u907z00033b69dt8230c6.jpg
7	ckl0uczwm00013669r8i9srq3.jpg
8	ckl0udhy800043b69kqqz85gb.jpg
9	ckl0udjnw00003b6d8ctd4nrb.jpg
10	ckl0uh9lw00013b694tuvpoxq.jpg
11	ckl0uic9y00053b69mw2hyi1d.jpg
12	ckl0ujgjy00023669nkipux5t.jpg
13	ckl12f1e200003b69qpdzdlmp.jpg
14	ckl12ihy400013b69785orwgc.jpg
15	ckl12l5o400023b690wyrf35l.jpg
16	ckl12nyb800033b690yyjdox1.jpg
17	ckl13h69800043b69988tvosu.jpg
18	ckl13lc0v00053b69ni5ed2cg.jpg
19	ckl13nxxd00063b69aj29f62k.jpg
20	ckl13qt3y00073b69v0xum2qr.jpg
21	ckl13tlkv00083b69qyuocjkb.jpg
22	ckl13xfum00093b69xzw5fyq6.jpg
23	ckl140n7l000a3b69dx7wc9n7.jpg
24	ckl142q7h000b3b69z7q2xk44.jpg
25	ckl146dkr000c3b69slllz6xe.jpg
26	ckl1498rq000d3b69x5f3ummb.jpg
27	ckl14jqyy00003b6985pwkig0.jpg
28	ckl14lbg500013b69z2m1s7mb.jpg
29	ckl14nf4u00023b69pa12bcvi.jpg
30	ckl14pccy00033b69

In [25]:
!cp -r /content/crossway/sample_data/dataset/res/* /content/drive/MyDrive/crossway/sample_data/dataset/res
!cp -r /content/crossway/ckpts/* /content/drive/MyDrive/crossway/ckpts

In [16]:
import cv2, time
from matplotlib import pyplot as plt
from threading import Thread
from IPython import display

In [12]:
class ThreadedCamera(object):
  def __init__(self, src):
    self.capture = cv2.VideoCapture(src)
    self.capture.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    self.status = False

    self.FPS = 1/30
    self.FPS_MS = int(self.FPS * 1000)

    self.thread = Thread(target=self.update, args=())
    self.thread.daemon = True
    self.thread.start()

  def update(self):
    while True:
      if self.capture.isOpened():
        (self.status, self.frame) = self.capture.read()
      time.Sleep(self.FPS)
  
  def get_frame(self):
    if self.status:
      return self.frame

In [18]:
src = 'https://sochi.camera:8081/cam_274/tracks-v1/mono.m3u8'
cap = ThreadedCamera(src)
while True:
  frame = cap.get_frame()
  if frame is not None:
    display.clear_output(wait=True)
    plt.imshow(frame)
    plt.show()
    cv2.waitKey(cap.FPS_MS)

KeyboardInterrupt: ignored

Label images

In [ ]:
def label_images(model, image_dir, obj_class, height, width):
  imgs = os.listdir(image_dir)
  labels = []
  for idx, filename in enumerate(sorted(imgs)):
    img = np.array(Image.open(image_dir+"/"+filename), dtype=np.float32)
    img = (img / 127.5) - 1.0
    prediction = expit(model.predict(np.expand_dims(img, 0))[0])
    centroids = centroids_of_connected_components(prediction, rescale=2.0)
    bitmap = xys_to_bitmap(centroids, height, width, 0.5)
    single_chan_img = bitmap_to_single_channel_pil_image(bitmap)
    single_chan_img.save("%s/%s_%s.png" % (image_dir, filename, obj_class))
    print("\t".join(map(str, [idx, filename, len(centroids)])))

In [ ]:
model = restore_model()
label_images(model, , obj, height, width)